# Vertex AI - External Model Serving

This notebook uses a Vertex AI Notebook to train a TensorFlow model (locally) with the data in BigQuery table `<PROJECT_ID>.digits.digits_prepped`.  This model is then saved and Vertex AI clients are used to upload the model and deploy it to an endpoint for online predictions.

**Prerequisites**
- `00 - Initial Setup`
- `01 - BigQuery - Data`

**Overview**

<img src="architectures/statmike-mlops-04.png">

---
## Setup

Prepare TensorFlow:

In [1]:
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession
import tensorflow as tf

Setup Parameters

In [2]:
PROJECT_ID='statmike-mlops'
REGION='us-central1'

BQDATASET_ID='digits'
BQTABLE_ID='digits_prepped'

MODEL_DIR='gs://{}/digits/model/04_keras'.format(PROJECT_ID)
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

BATCH_SIZE = 30

MODEL_NAME='MODEL_04_KERAS-DIGITS'
ENDPOINT_NAME='ENDPOINT_04_KERAS-DIGITS'
params = {"MODEL_DIR":MODEL_DIR}
DEPLOY_IMAGE='us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-2:latest'
DEPLOY_COMPUTE='n1-standard-4'

Setup Vertex AI Python Clients
- https://googleapis.dev/python/aiplatform/latest/index.html

In [3]:
from google.cloud import aiplatform

API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
client_options = {"api_endpoint": API_ENDPOINT}
clients = {}

---
## Prepare Data Connection

Retrieve the Schema info from BigQuery Information Schema via the Storage API:
- https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas

In [4]:
from google.cloud import bigquery
bqclient = bigquery.Client()
bqjob = bqclient.query(
"""
SELECT * FROM `"""+BQDATASET_ID+""".INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
WHERE TABLE_NAME = '"""+BQTABLE_ID+"""' """
)
schema = bqjob.result().to_dataframe()
schema

,table_catalog,table_schema,table_name,column_name,field_path,data_type,description
0,statmike-mlops,digits,digits_prepped,p0,p0,FLOAT64,None
1,statmike-mlops,digits,digits_prepped,p1,p1,FLOAT64,None
2,statmike-mlops,digits,digits_prepped,p2,p2,FLOAT64,None
3,statmike-mlops,digits,digits_prepped,p3,p3,FLOAT64,None
4,statmike-mlops,digits,digits_prepped,p4,p4,FLOAT64,None
...,...,...,...,...,...,...,...
62,statmike-mlops,digits,digits_prepped,p62,p62,FLOAT64,None
63,statmike-mlops,digits,digits_prepped,p63,p63,FLOAT64,None
64,statmike-mlops,digits,digits_prepped,target,target,INT64,None
65,statmike-mlops,digits,digits_prepped,target_OE,target_OE,STRING,None


Use the the table schema to prepare the TensorFlow Model:
- Omit unused columns
- Create `feature_columns` for the model
- Define the `dtypes` for TensorFlow

In [5]:
OMIT = ['target_OE','SPLITS']

selected_fields = schema[~schema.column_name.isin(OMIT)].column_name.tolist()

feature_columns = []
feature_layer_inputs = {}
for header in selected_fields:
    if header != 'target':
        feature_columns.append(tf.feature_column.numeric_column(header))
        feature_layer_inputs[header] = tf.keras.Input(shape=(1,),name=header)

from tensorflow.python.framework import dtypes
output_types = schema[~schema.column_name.isin(OMIT)].data_type.tolist()
output_types = [dtypes.float64 if x=='FLOAT64' else dtypes.int64 for x in output_types]

Define a function that remaps the input data for TensorFlow into features, target and one_hot encodes the `target`:

In [6]:
def transTable(row_dict):
    target=row_dict.pop('target')
    target = tf.one_hot(tf.cast(target,tf.int64),10)
    target = tf.cast(target,tf.float32)
    return(row_dict,target)

Setup TensorFlow_IO client > session > table + table.map
- https://www.tensorflow.org/io/api_docs/python/tfio/bigquery/BigQueryClient

In [7]:
client = BigQueryClient()
session = client.read_session("projects/"+PROJECT_ID,PROJECT_ID,BQTABLE_ID,BQDATASET_ID,selected_fields,output_types,row_restriction="SPLITS='TRAIN'",requested_streams=3)
table = session.parallel_read_rows()
table = table.map(transTable)
train = table.shuffle(100000).batch(BATCH_SIZE)

In [8]:
client = BigQueryClient()
session = client.read_session("projects/"+PROJECT_ID,PROJECT_ID,BQTABLE_ID,BQDATASET_ID,selected_fields,output_types,row_restriction="SPLITS='TEST'",requested_streams=3)
table = session.parallel_read_rows()
table = table.map(transTable)
test = table.batch(BATCH_SIZE)

Review a single batch of the train data:

In [9]:
for a, b in train.take(1):
    columns=list(a.keys())
    print('columns: ',columns)
    print('target: ',b)

columns:  ['p0', 'p1', 'p10', 'p11', 'p12', 'p13', 'p14', 'p15', 'p16', 'p17', 'p18', 'p19', 'p2', 'p20', 'p21', 'p22', 'p23', 'p24', 'p25', 'p26', 'p27', 'p28', 'p29', 'p3', 'p30', 'p31', 'p32', 'p33', 'p34', 'p35', 'p36', 'p37', 'p38', 'p39', 'p4', 'p40', 'p41', 'p42', 'p43', 'p44', 'p45', 'p46', 'p47', 'p48', 'p49', 'p5', 'p50', 'p51', 'p52', 'p53', 'p54', 'p55', 'p56', 'p57', 'p58', 'p59', 'p6', 'p60', 'p61', 'p62', 'p63', 'p7', 'p8', 'p9']
target:  tf.Tensor(
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0

---
## Train the Model

Define the Model:

In [10]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
feature_layer_outputs = feature_layer(feature_layer_inputs)
model = tf.keras.Model(inputs=[v for v in feature_layer_inputs.values()],outputs=tf.keras.layers.Dense(10,activation=tf.nn.softmax)(feature_layer_outputs))
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
#tf.keras.utils.plot_model(model,show_shapes=True, show_dtype=True)

In [11]:
#model.summary()

Fit the Model:

In [12]:
history = model.fit(train,epochs=25)

Epoch 1/25
48/48 [==============================] - 2s 4ms/step - loss: 3.6315 - accuracy: 0.4927
Epoch 2/25
48/48 [==============================] - 0s 3ms/step - loss: 0.7070 - accuracy: 0.8050
Epoch 3/25
48/48 [==============================] - 0s 4ms/step - loss: 0.3959 - accuracy: 0.8826
Epoch 4/25
48/48 [==============================] - 0s 4ms/step - loss: 0.3006 - accuracy: 0.9078
Epoch 5/25
48/48 [==============================] - 0s 4ms/step - loss: 0.2456 - accuracy: 0.9245
Epoch 6/25
48/48 [==============================] - 0s 4ms/step - loss: 0.2101 - accuracy: 0.9350
Epoch 7/25
48/48 [==============================] - 0s 4ms/step - loss: 0.1942 - accuracy: 0.9392
Epoch 8/25
48/48 [==============================] - 1s 4ms/step - loss: 0.1546 - accuracy: 0.9518
Epoch 9/25
48/48 [==============================] - 0s 3ms/step - loss: 0.1467 - accuracy: 0.9497
Epoch 10/25
48/48 [==============================] - 0s 3ms/step - loss: 0.1295 - accuracy: 0.9560
Epoch 11/25
48/48 [

Evaluate the model with the test data:

In [13]:
loss, accuracy = model.evaluate(test)

13/13 [==============================] - 1s 5ms/step - loss: 0.1354 - accuracy: 0.9563


Create Prediction from a batch of the test data:

In [14]:
model.predict(test.take(1))

array([[9.98126924e-01, 7.77293026e-05, 3.10619939e-08, 2.52564081e-11,
        6.47546840e-04, 2.93015682e-06, 9.16230143e-04, 8.30095885e-08,
        2.18975270e-04, 9.45235024e-06],
       [9.99773443e-01, 1.94345073e-08, 9.40447720e-09, 5.47635617e-11,
        6.30679040e-07, 1.78677801e-08, 7.65936704e-07, 7.42058748e-10,
        1.78904847e-05, 2.07195975e-04],
       [9.99573767e-01, 5.32874855e-10, 4.61138995e-07, 4.77575444e-11,
        5.19682146e-08, 7.83407870e-07, 3.25622794e-04, 1.32466116e-09,
        8.18743592e-06, 9.10424278e-05],
       [8.18498492e-01, 5.04008145e-04, 6.64339723e-06, 8.22219862e-11,
        1.81530166e-04, 1.13334590e-05, 1.76574677e-01, 5.27156317e-06,
        4.13569948e-03, 8.23972878e-05],
       [9.37826931e-01, 5.85520524e-04, 6.72940905e-06, 1.10499665e-11,
        6.11661263e-02, 4.65437289e-08, 1.25485531e-04, 2.62845017e-04,
        2.61703062e-05, 7.14297386e-08],
       [9.99708354e-01, 2.34189869e-07, 2.24868613e-09, 1.10485392e-10,
   

---
## Save the model:

In [15]:
model.save(MODEL_DIR)

INFO:tensorflow:Assets written to: gs://statmike-mlops/digits/model/04_keras/assets
INFO:tensorflow:Assets written to: gs://statmike-mlops/digits/model/04_keras/assets


---
## Upload the Model to Vertex AI

Create a client to the Model Service, define the Model, and upload the model:

In [16]:
clients['model'] = aiplatform.gapic.ModelServiceClient(client_options=client_options)

MODEL = {
    "display_name": MODEL_NAME,
    "metadata_schema_uri": "",
    "artifact_uri": MODEL_DIR,
    "container_spec": {
        "image_uri": DEPLOY_IMAGE,
        "command": [],
        "args": [],
        "env": [],
        "ports": [{"container_port": 8080}],
        "predict_route": "",
        "health_route": ""
    }
}

uploaded_model = clients['model'].upload_model(parent=PARENT, model=MODEL)

Retrieve the model information and view the name and display name:

In [17]:
model_info = clients['model'].get_model(name=uploaded_model.result(timeout=180).model)
model_info.display_name, model_info.name

('MODEL_04_KERAS-DIGITS',
 'projects/691911073727/locations/us-central1/models/6816629044583333888')

---
## Create the Vertex AI Endpoint

Create a client to the Endpoint Service and use it to create the endpoint:

In [18]:
clients['endpoint'] = aiplatform.gapic.EndpointServiceClient(client_options=client_options)

endpoint = clients['endpoint'].create_endpoint(parent=PARENT, endpoint={"display_name": ENDPOINT_NAME})

Retrieve the endpoint information and view the name and display name:

In [19]:
endpoint_info = clients['endpoint'].get_endpoint(name=endpoint.result(timeout=180).name)
endpoint_info.display_name, endpoint_info.name

('ENDPOINT_04_KERAS-DIGITS',
 'projects/691911073727/locations/us-central1/endpoints/8251913931296079872')

---
## Deploy the Model to the Vertex AI Endpoint

In [20]:
DMODEL = {
        "model": model_info.name,
        "display_name": 'DEPLOYED_'+MODEL_NAME,
        "dedicated_resources": {
            "min_replica_count": 1,
            "max_replica_count": 1,
            "machine_spec": {
                    "machine_type": DEPLOY_COMPUTE,
                    "accelerator_count": 0,
                }
        }   
}

TRAFFIC = {
    '0' : 100
}

dmodel = clients['endpoint'].deploy_model(endpoint=endpoint_info.name, deployed_model=DMODEL, traffic_split=TRAFFIC)

Retrieve the deployed model information from the endpoint:

In [22]:
clients['endpoint'].get_endpoint(name=endpoint_info.name).deployed_models

[id: "2957158513136107520"
model: "projects/691911073727/locations/us-central1/models/6816629044583333888"
display_name: "DEPLOYED_MODEL_04_KERAS-DIGITS"
create_time {
  seconds: 1625580314
  nanos: 43208000
}
dedicated_resources {
  machine_spec {
    machine_type: "n1-standard-4"
  }
  min_replica_count: 1
  max_replica_count: 1
}
]

---
## Predictions

Create a client to the prediction service:

In [4]:
clients['prediction'] = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

Setup an observation for prediction:

In [5]:
%%bigquery pred
SELECT *
FROM `digits.digits_prepped`
WHERE splits='TEST'

Downloading: 100%|██████████| 366/366 [00:01<00:00, 335.41rows/s]


In [6]:
pred.head(1)

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p57,p58,p59,p60,p61,p62,p63,target,target_OE,SPLITS
0,0.0,0.0,0.0,9.0,8.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,7.0,14.0,9.0,0.0,0.0,0,Even,TEST


In [8]:
newob = pred.loc[:0,'p0':'p63'].to_dict(orient='records')[0]
#newob

### With Python Client

Request prediction from the prediction service:

In [27]:
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

response = clients['prediction'].predict(endpoint=endpoint_info.name, instances=[json_format.ParseDict(newob, Value())], parameters=json_format.ParseDict({}, Value()))

In [28]:
response.predictions

[[0.998126924, 7.77293e-05, 3.10619903e-08, 2.52563109e-11, 0.000647545618, 2.93015682e-06, 0.000916230143, 8.30095885e-08, 0.00021897527, 9.45234115e-06]]

In [29]:
import numpy as np
np.argmax(response.predictions[0])

0

### With REST

In [30]:
import json
with open('request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [32]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @request.json \
https://{API_ENDPOINT}/v1/{endpoint_info.name}:predict

{
  "predictions": [
    [
      0.998126924,
      7.77293e-05,
      3.10619903e-08,
      2.52563109e-11,
      0.000647545618,
      2.93015682e-06,
      0.000916230143,
      8.30095885e-08,
      0.00021897527,
      9.45234115e-06
    ]
  ],
  "deployedModelId": "2957158513136107520"
}


### With gcloud

In [34]:
!gcloud beta ai endpoints predict {endpoint_info.name.rsplit('/',1)[-1]} --region={REGION} --json-request=request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[[0.998126924, 7.77293e-05, 3.10619903e-08, 2.52563109e-11, 0.000647545618, 2.93015682e-06, 0.000916230143, 8.30095885e-08, 0.00021897527, 9.45234115e-06]]


# Remove Resources
- undeploy-model
- remove endpoint
- remove model
- delete model files

Undeploy Model:

In [79]:
dmodel = clients['endpoint'].get_endpoint(name=endpoint_info.name).deployed_models[0].id
clients['endpoint'].undeploy_model(endpoint=endpoint_info.name, deployed_model_id=dmodel)

Delete Endpoint:

In [80]:
clients['endpoint'].delete_endpoint(name=endpoint_info.name)

Remove Model:

In [81]:
clients['model'].delete_model(name=model_info.name)

Delete Model Files:

In [82]:
from google.cloud import storage
gcs = storage.Client()

path = gcs.bucket(PROJECT_ID)
blobs = path.list_blobs(prefix='digits/keras')
for blob in blobs:
    blob.delete()